<a href="https://colab.research.google.com/github/satishsurath/OpenAIfinancialAdvisorEmbeddings/blob/main/Building_a_Financial_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#gpu_info = !nvidia-smi
#gpu_info = '\n'.join(gpu_info)
#if gpu_info.find('failed') >= 0:
#  print('Not connected to a GPU')
#else:
#  print(gpu_info)

import subprocess

try:
    gpu_info = subprocess.check_output(['system_profiler', 'SPDisplaysDataType']).decode('utf-8')
except subprocess.CalledProcessError as e:
    gpu_info = str(e.output)

if 'GPU' not in gpu_info and 'Graphics' not in gpu_info:
    print('No GPU information available')
else:
    print(gpu_info)

Graphics/Displays:

    Apple M2 Max:

      Chipset Model: Apple M2 Max
      Type: GPU
      Bus: Built-In
      Total Number of Cores: 30
      Vendor: Apple (0x106b)
      Metal Support: Metal 3
      Displays:
        Color LCD:
          Display Type: Built-in Liquid Retina XDR Display
          Resolution: 3024 x 1964 Retina
          Main Display: Yes
          Mirror: Off
          Online: Yes
          Automatically Adjust Brightness: No
          Connection Type: Internal




In [5]:
!pip install openai pytube -q
!pip install git+https://github.com/openai/whisper.git -q

In [34]:
import openai
import whisper
import pandas as pd
import re
#from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from getpass import getpass

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"

In [35]:
openai.api_key = getpass("Enter your OpenAI API Key")

Enter your OpenAI API Key ········


Adding some more Terminal Code to get this running:

In [36]:
model = whisper.load_model('base')

In [37]:
youtube_video_url = "https://www.youtube.com/watch?v=XZg_iuGN3AA"

In [38]:

# extract video id from youtube url
def extract_video_id(url):
    pattern = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/watch\?v=|youtu\.be\/)([\w-]{11})'
    match = re.match(pattern, url)
    if match:
        return match.group(1)
    return None

video_id = extract_video_id(youtube_video_url)
print(video_id)

XZg_iuGN3AA


In [39]:
transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
transcript_text = ' '.join([item['text'] for item in transcript_list])
print(transcript_text)

hey everyone Peter Zion here coming to you from Colorado where spring has sprung and the frogs are chirping which means of course later today we're supposed to get a foot of snow anyway uh I thought it would be worth me commenting on the recent intelligence leaks by Airmen uh to Shara let's deal with the contents of what was leaked and then we can talk about Espionage and leaks in general so most of the documents that were leaked relate to the war in Ukraine in some way and involve internal U.S assessments of how the war is going and how the ukrainians are doing and they're broadly less than fully complementary uh basic indication from the leak is that the ukrainians have been suffering higher casualties than are reported uh and more importantly that the confidence in the numbers provided by the ukrainians is very low so the US really doesn't have a good view in addition there's concern that by engaging in a static defense in places like bakluth the ukrainians are losing their combat F

In [ ]:
import csv

# create a new pandas dataframe
df = pd.DataFrame(columns=['episode', 'url', 'start_timestamp', 'start', 'end', 'question', 'context'])

# index to keep track of current row
i = 0

with open('financial_advisor.csv') as csv_file:
    reader = csv.reader(csv_file)

    # skip the header row in the csv file
    next(reader)
    
    for row in reader:
        # assign each column in the row to a variable and split questions on carriage return
        episode, url, questions = row
        question_list = questions.split("\n")

        # for each question in the list, extract the timestamp and convert it to seconds for youtube
        for question in question_list:
          pieces = question.split('-')
          timestamp = pieces[0]
          minutes, seconds = timestamp.split(':')
          seconds = int(seconds) + (int(minutes.lstrip()) * 60)

          # add a new row to the dataframe
          df.loc[i] = [episode, url, timestamp, seconds, seconds, " ".join(pieces[1:]), ""]
          
          try:
            df.loc[i-1]['end'] = df.loc[i]['start']
          except:
            print(f"skipping row {i} because there is no previous row")

          i+=1

          df['end'][df['end'] < df['start']] = 0
          df['end'][df['start'] == df['end']] = 0

df.to_csv("questions.csv")


skipping row 0 because there is no previous row


In [40]:
import pandas as pd

df = pd.read_csv('questions.csv')
df

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
0,0,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,3:07,187,415,What's the best way to reallocate a portfolio ...,NaN
1,1,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,6:55,415,669,What do I do once I cross the Roth IRA income ...,NaN
2,2,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,11:09,669,0,Once I have maxed out my retiremenet accounts...,NaN
3,3,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,1:04,64,366,Saving vs. Investing,NaN
4,4,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,6:06,366,566,If Baby Boomers are shifting from net investo...,NaN
...,...,...,...,...,...,...,...,...
293,293,Episode 61,https://www.youtube.com/watch?v=dYQAYc26bm0,2:56,176,443,Adding bonds to a 100% stock portfolio,NaN
294,294,Episode 61,https://www.youtube.com/watch?v=dYQAYc26bm0,7:23,443,670,Is having a pension and annuity a substitute f...,NaN
295,295,Episode 61,https://www.youtube.com/watch?v=dYQAYc26bm0,11:10,670,1103,Can the stock market grow forever?,NaN
296,296,Episode 61,https://www.youtube.com/watch?v=dYQAYc26bm0,18:23,1103,1698,Whole life insurance.,NaN


In [41]:
# Let's just get the questions for a single episode and make this work before we download and transcribe all episodes in bulk
episode = df[df['episode'] == 'Episode 22'].copy()
episode

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
83,83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,1:58,118,210,How does the fed raising interest rates fights...,NaN
84,84,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,3:30,210,316,What particular sectors/names do you find most...,NaN
85,85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,5:16,316,618,What will bring more value in the long run for...,NaN
86,86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,717,Would you recommend focusing on paying off my ...,NaN
87,87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,805,Paying off a mortgage.,NaN
88,88,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,13:25,805,879,What happens when the Fed raises rates?,NaN
89,89,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,14:39,879,1039,I’m 44 and have a net worth of around $425K. ...,NaN
90,90,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,17:19,1039,1275,What are some good questions to ask when sele...,NaN
91,91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...",NaN


In [42]:
def is_part_of_question(segment, start, end):
    if segment['start'] > start:
      if segment['end'] < end or end == 0:
          return True 

    return False

def get_question_context(row):
     global output

     question_segments = list(filter(lambda segment: is_part_of_question(segment, row['start'], row['end']), output['segments']))
     # include question from timestamp in the context
     context = row['question']
     for segment in question_segments:
         context += segment['text']

     return context

episode['context'] = episode.apply(get_question_context, axis=1)

episode

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
83,83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,1:58,118,210,How does the fed raising interest rates fights...,How does the fed raising interest rates fights...
84,84,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,3:30,210,316,What particular sectors/names do you find most...,What particular sectors/names do you find most...
85,85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,5:16,316,618,What will bring more value in the long run for...,What will bring more value in the long run for...
86,86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,717,Would you recommend focusing on paying off my ...,Would you recommend focusing on paying off my ...
87,87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,805,Paying off a mortgage.,"Paying off a mortgage. OK, so up next, we have..."
88,88,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,13:25,805,879,What happens when the Fed raises rates?,"What happens when the Fed raises rates? OK, so..."
89,89,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,14:39,879,1039,I’m 44 and have a net worth of around $425K. ...,I’m 44 and have a net worth of around $425K. ...
90,90,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,17:19,1039,1275,What are some good questions to ask when sele...,What are some good questions to ask when sele...
91,91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...","At interest rates so low, should I instead t..."


In [43]:
from openai.embeddings_utils import get_embedding

#get_embedding(episode.iloc[0]['context'], engine='text-embedding-ada-002')

episode['embedding'] = episode['context'].apply(lambda row: get_embedding(row, engine='text-embedding-ada-002'))
episode.to_csv('question_embeddings.csv')

In [50]:
from openai.embeddings_utils import cosine_similarity

question = "How much should I save for retirement"
question_vector = get_embedding(question, engine='text-embedding-ada-002')

episode["similarities"] = episode['embedding'].apply(lambda x: cosine_similarity(x, question_vector))
episode = episode.sort_values("similarities", ascending=False).head(4)

episode

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context,embedding,similarities
86,86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,717,Would you recommend focusing on paying off my ...,Would you recommend focusing on paying off my ...,"[-0.006371411494910717, -0.01449133176356554, ...",0.814878
85,85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,5:16,316,618,What will bring more value in the long run for...,What will bring more value in the long run for...,"[0.022100025787949562, -0.004235513508319855, ...",0.806475
87,87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,805,Paying off a mortgage.,"Paying off a mortgage. OK, so up next, we have...","[-0.017217114567756653, 0.01616808958351612, 0...",0.800483
91,91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...","At interest rates so low, should I instead t...","[-0.0116172069683671, -0.014779669232666492, 0...",0.780772


In [51]:
episode.to_csv("sorted.csv")

In [52]:
context = []
for i, row in episode.iterrows():
  context.append(row['context'])

context

["Would you recommend focusing on paying off my student loans first? Or should I begin building my portfolio? The surprise is over. OK, so I have next. Long time viewer of the show, what all the content you put out. I'm 23 and just graduated from college. I landed my first job at a financial services firm. And I'm wondering how I should allocate my earnings. Would you recommend focusing on paying off my student loans first around $30,000 or building up my retirement portfolio? My boring answer to this one is I like the idea of splitting it up because I think that's the simplest way to go about it. And I think building those financial habits early and saving important. Now, it could be one of these people that just loves paying off debt because you hate it so much. Duncan, I can't remember. You a Duke or North Carolina fan because you're from North Carolina. I'm a Duke fan. OK, so some people hate debt as much as people outside of Durham, North Carolina hate coach K and Duke, right? Oh,

In [53]:
text = "\n".join(context)
text

"Would you recommend focusing on paying off my student loans first? Or should I begin building my portfolio? The surprise is over. OK, so I have next. Long time viewer of the show, what all the content you put out. I'm 23 and just graduated from college. I landed my first job at a financial services firm. And I'm wondering how I should allocate my earnings. Would you recommend focusing on paying off my student loans first around $30,000 or building up my retirement portfolio? My boring answer to this one is I like the idea of splitting it up because I think that's the simplest way to go about it. And I think building those financial habits early and saving important. Now, it could be one of these people that just loves paying off debt because you hate it so much. Duncan, I can't remember. You a Duke or North Carolina fan because you're from North Carolina. I'm a Duke fan. OK, so some people hate debt as much as people outside of Durham, North Carolina hate coach K and Duke, right? Oh, 

In [54]:
context = text

context

"Would you recommend focusing on paying off my student loans first? Or should I begin building my portfolio? The surprise is over. OK, so I have next. Long time viewer of the show, what all the content you put out. I'm 23 and just graduated from college. I landed my first job at a financial services firm. And I'm wondering how I should allocate my earnings. Would you recommend focusing on paying off my student loans first around $30,000 or building up my retirement portfolio? My boring answer to this one is I like the idea of splitting it up because I think that's the simplest way to go about it. And I think building those financial habits early and saving important. Now, it could be one of these people that just loves paying off debt because you hate it so much. Duncan, I can't remember. You a Duke or North Carolina fan because you're from North Carolina. I'm a Duke fan. OK, so some people hate debt as much as people outside of Durham, North Carolina hate coach K and Duke, right? Oh, 

In [55]:
prompt = f"""Answer the following question using only the context below. Answer in the style of Ben Carlson a financial advisor and podcaster. If you don't know the answer for certain, say I don't know.

Context:
{context}

Q: {question}
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=1,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Ultimately, how much you save for retirement will depend on your individual circumstances. That said, in general, it's recommended that you save at least 10-15% of your gross income. Depending on where you are right now, you may need to increase your savings rate to hit your future retirement goal."